In [1]:
import polars as pl

In [2]:
#lets read in the data I downloaded from hugging face, I start with the last 6 files
df1 = pl.read_parquet("train-00121-of-00127.parquet")
df2 = pl.read_parquet("train-00122-of-00127.parquet")
df3 = pl.read_parquet("train-00123-of-00127.parquet")
df4 = pl.read_parquet("train-00124-of-00127.parquet")
df5 = pl.read_parquet("train-00125-of-00127.parquet")
df6 = pl.read_parquet("train-00126-of-00127.parquet")

In [3]:
# let's filter out issues without pull requests
onlyPR1 = df1.filter(pl.col("pull_request")!="{null,null,null}")
onlyPR2 = df2.filter(pl.col("pull_request")!="{null,null,null}")
onlyPR3 = df3.filter(pl.col("pull_request")!="{null,null,null}")
onlyPR4 = df4.filter(pl.col("pull_request")!="{null,null,null}")
onlyPR5 = df5.filter(pl.col("pull_request")!="{null,null,null}")
onlyPR6 = df6.filter(pl.col("pull_request")!="{null,null,null}")

In [4]:
# let's squash it into one df
pr_df = pl.concat([onlyPR1,onlyPR2,onlyPR3,onlyPR4,onlyPR5,onlyPR6])

In [5]:
# that's a pretty big nice dataset
pr_df.shape

(478525, 8)

In [6]:
# to compare the Open Issues to the issues from our dataset, we extract what's between
# the first <issue comment> and second <issue comment> and save it in "issue" column
# since that's the title and the description of the issue and will be crucial in estimating the similarity between
# Open issues and issue from our dataset
pr_df

repo,issue_id,issue_number,pull_request,events,text_size,content,usernames
str,i64,i64,struct[3],list[struct[8]],i64,str,str
"""kaisermann/svelte-i18n""",550510104,40,"{40,""svelte-i18n"",""kaisermann""}","[{""opened"",""kaisermann"",null,""2020-01-16T00:48:55Z"",""username_0"","""",""WIP - v3"",""issue""}, {""created"",""elbourki"",580035734,""2020-01-30 01:00:41+00:00"",""username_1"",""Hey! That's a pretty awesome package you got there! What if we removed `getClientLocale` from utils and made it 100% optional? We could do that by making `initialLocale` accept an array of _ISO_ codes and it finds the first one that has registered translations... I think this way it would allow developers to use external libraries like _wojtekmaj/get-user-locale_, they offer a pretty handy method called `getUserLocales` that returns an array as well, so this means you won't have to maintain that functionality and developers would have more flexibility. And please let me know if you have a to do list or something, I can help with some tasks :)"",null,""comment""}, … {""created"",""elbourki"",586596649,""2020-02-15 14:40:59+00:00"",""username_1"",""Hey Chris! I apologize for not being able to help with 3.0, I got a bit busier in the last few weeks :( I was thinking of replacing `intl-messageformat` with `message-format`, it's at 21.5kB compared to 29.8kB, not a huge improvement but it would reduce the library size to 28.4kB 😬 But note that they don't have the same [API](https://github.com/format-message/format-message/tree/master/packages/message-format#messageformat), we'd have to remove the _formats_ option if we ever choose to adopt this library. Another interesting option would be [Frenchkiss.js](https://github.com/koala-interactive/frenchkiss.js), it supports ICU and it's currently at 2.7kB 😱. The way the library works is quite different and we will probably need a major rewrite to implement it, but I think it's worth it :D"",null,""comment""}]",2398,"""<issue_start><issue_comment>Ti…","""[""kaisermann"", ""elbourki""]"""
"""material-components/material-c…",551064006,9444,"{9444,""material-components-ios"",""material-components""}","[{""opened"",""jverkoey"",null,""2020-01-16T21:25:40Z"",""username_0"",""[AppBar] Fix swipe to go back gesture for MDCAppBarNavigationController. https://developer.apple.com/design/human-interface-guidelines/ios/user-interaction/gestures/ ""But users can also navigate back by swiping from the side of the screen."""",""[AppBar] Fix swipe to go back gesture for MDCAppBarNavigationController."",""issue""}, {""created"",""bryanoltman"",575356739,""2020-01-16 21:32:17+00:00"",""username_1"",""Did you mean to merge this into `develop`?"",null,""comment""}, {""created"",""jverkoey"",575357719,""2020-01-16 21:34:52+00:00"",""username_0"",""Yep; will update once ready for proper review. Fixing attributes first."",null,""comment""}]",355,"""<issue_start><issue_comment>Ti…","""[""jverkoey"", ""bryanoltman""]"""
"""hmcts/ccd-data-store-api""",551466679,758,"{758,""ccd-data-store-api"",""hmcts""}","[{""opened"",""noronhaa"",null,""2020-01-17T15:06:13Z"",""username_0"",""**Before creating a pull request make sure that:** - [ ] commit messages are meaningful and follow good commit message guidelines - [ ] README and other documentation has been updated / added (if needed) - [ ] tests have been updated / new tests has been added (if needed) Please remove this line and everything above and fill the following sections: ### JIRA link (if applicable) ### https://tools.hmcts.net/jira/browse/RDM-7289 ### Change description ### Update jackson-databind **Does this PR introduce a breaking change?** (check one with ""x"") ``` [ ] Yes [x] No ```"",""RDM-7289 - Update jackson-databind"",""issue""}]",601,"""<issue_start><issue_comment>Ti…","""[""noronhaa""]"""
"""dlang/phobos""",551980198,7355,"{7355,""phobos"",""dlang""}","[{""opened"",""jacob-carlborg"",null,""2020-01-19T20:32:25Z"",""username_0"",""I've only tested this on a 64 bit iPho

In [7]:
# regex on friday night, yeah!
import re

def extract_text(text): 
    pattern = r'<issue_comment>(.*?)<issue_comment>' 
    match = re.search(pattern, text, re.DOTALL) 
    if match: 
        return match.group(1) 
    else: 
        return None

In [8]:
pr_df = pr_df.with_columns([ pl.col("content").map_elements(extract_text, return_dtype=pl.Utf8).alias("issue") ])

In [9]:
pr_df.head()
#and it worked!

repo,issue_id,issue_number,pull_request,events,text_size,content,usernames,issue
str,i64,i64,struct[3],list[struct[8]],i64,str,str,str
"""kaisermann/svelte-i18n""",550510104,40,"{40,""svelte-i18n"",""kaisermann""}","[{""opened"",""kaisermann"",null,""2020-01-16T00:48:55Z"",""username_0"","""",""WIP - v3"",""issue""}, {""created"",""elbourki"",580035734,""2020-01-30 01:00:41+00:00"",""username_1"",""Hey! That's a pretty awesome package you got there! What if we removed `getClientLocale` from utils and made it 100% optional? We could do that by making `initialLocale` accept an array of _ISO_ codes and it finds the first one that has registered translations... I think this way it would allow developers to use external libraries like _wojtekmaj/get-user-locale_, they offer a pretty handy method called `getUserLocales` that returns an array as well, so this means you won't have to maintain that functionality and developers would have more flexibility. And please let me know if you have a to do list or something, I can help with some tasks :)"",null,""comment""}, … {""created"",""elbourki"",586596649,""2020-02-15 14:40:59+00:00"",""username_1"",""Hey Chris! I apologize for not being able to help with 3.0, I got a bit busier in the last few weeks :( I was thinking of replacing `intl-messageformat` with `message-format`, it's at 21.5kB compared to 29.8kB, not a huge improvement but it would reduce the library size to 28.4kB 😬 But note that they don't have the same [API](https://github.com/format-message/format-message/tree/master/packages/message-format#messageformat), we'd have to remove the _formats_ option if we ever choose to adopt this library. Another interesting option would be [Frenchkiss.js](https://github.com/koala-interactive/frenchkiss.js), it supports ICU and it's currently at 2.7kB 😱. The way the library works is quite different and we will probably need a major rewrite to implement it, but I think it's worth it :D"",null,""comment""}]",2398,"""<issue_start><issue_comment>Ti…","""[""kaisermann"", ""elbourki""]""","""Title: WIP - v3 username_0: """
"""material-components/material-c…",551064006,9444,"{9444,""material-components-ios"",""material-components""}","[{""opened"",""jverkoey"",null,""2020-01-16T21:25:40Z"",""username_0"",""[AppBar] Fix swipe to go back gesture for MDCAppBarNavigationController. https://developer.apple.com/design/human-interface-guidelines/ios/user-interaction/gestures/ ""But users can also navigate back by swiping from the side of the screen."""",""[AppBar] Fix swipe to go back gesture for MDCAppBarNavigationController."",""issue""}, {""created"",""bryanoltman"",575356739,""2020-01-16 21:32:17+00:00"",""username_1"",""Did you mean to merge this into `develop`?"",null,""comment""}, {""created"",""jverkoey"",575357719,""2020-01-16 21:34:52+00:00"",""username_0"",""Yep; will update once ready for proper review. Fixing attributes first."",null,""comment""}]",355,"""<issue_start><issue_comment>Ti…","""[""jverkoey"", ""bryanoltman""]""","""Title: [AppBar] Fix swipe to g…"
"""hmcts/ccd-data-store-api""",551466679,758,"{758,""ccd-data-store-api"",""hmcts""}","[{""opened"",""noronhaa"",null,""2020-01-17T15:06:13Z"",""username_0"",""**Before creating a pull request make sure that:** - [ ] commit messages are meaningful and follow good commit message guidelines - [ ] README and other documentation has been updated / added (if needed) - [ ] tests have been updated / new tests has been added (if needed) Please remove this line and everything above and fill the following sections: ### JIRA link (if applicable) ### https://tools.hmcts.net/jira/browse/RDM-7289 ### Change description ### Update jackson-databind **Does this PR introduce a breaking change?** (check one with ""x"") ``` [ ] Yes [x] No ```"",""RDM-7289 - Update jackson-databind"",""issue""}]",601,"""<issue_start><issue_comment>Ti…","""[""noronhaa""]""",null
"""dlang/phobos""",551980198,7355,"{7355,""phobos"",""dlang""}","[{""opened"",""jacob-carlborg""

In [10]:
# since I used <issue_comment> as the second delimiter it produced output only for Issues with at least one comment
# and there were some null values from issues with only titles and descriptions, so let's leave it like that
# and remove the null values
pr_df = pr_df.filter(pl.col("issue").is_not_null())
pr_df.shape

(414185, 9)

In [11]:
pr_df.columns

['repo',
 'issue_id',
 'issue_number',
 'pull_request',
 'events',
 'text_size',
 'content',
 'usernames',
 'issue']

In [12]:
# lets now split information in "repo" column into two separate columns, specifically "parent_repo" and "child_repo"
pr_df = pr_df.with_columns(pr_df.select(pl.col("repo").str.split(by="/").list.to_struct(n_field_strategy="max_width")).unnest("repo"))

In [13]:
pr_df = pr_df.rename({"field_0": "parent_repo", "field_1": "child_repo"})

In [14]:
pr_df["parent_repo"].value_counts(sort=True)

parent_repo,count
str,u32
"""apache""",8924
"""kubernetes""",4527
"""MicrosoftDocs""",4115
"""NixOS""",4103
"""Azure""",3799
…,…
"""webview""",1
"""DevShiftTeam""",1
"""geoffrich""",1


In [15]:
#let's filter for columns that we will use in the model
pr_df = pr_df.select(
    ['repo',
     'parent_repo',
     'child_repo',
     'issue_id',
     'issue_number',
     'issue',
     'text_size',
     'usernames',
    ])
pr_df

repo,parent_repo,child_repo,issue_id,issue_number,issue,text_size,usernames
str,str,str,i64,i64,str,i64,str
"""kaisermann/svelte-i18n""","""kaisermann""","""svelte-i18n""",550510104,40,"""Title: WIP - v3 username_0: """,2398,"""[""kaisermann"", ""elbourki""]"""
"""material-components/material-c…","""material-components""","""material-components-ios""",551064006,9444,"""Title: [AppBar] Fix swipe to g…",355,"""[""jverkoey"", ""bryanoltman""]"""
"""dlang/phobos""","""dlang""","""phobos""",551980198,7355,"""Title: Add initial support for…",4306,"""[""wilzbach"", ""etcimon"", ""Geod2…"
"""patternfly/patternfly-elements""","""patternfly""","""patternfly-elements""",552466427,686,"""Title: added fix for nav prior…",1995,"""[""starryeyez024"", ""LyndseyR""]"""
"""NixOS/nixpkgs""","""NixOS""","""nixpkgs""",553379763,78253,"""Title: coqPackages.CoLoR: 1.4.…",2544,"""[""vbgl"", ""jpas""]"""
…,…,…,…,…,…,…,…
"""okta/odyssey""","""okta""","""odyssey""",1183714195,1399,"""Title: Fix: Table sort indicat…",431,"""[""edburyenegren-okta"", ""jeffbe…"
"""rust-bitcoin/rust-bitcoin""","""rust-bitcoin""","""rust-bitcoin""",1188051396,924,"""Title: Improvements to taproot…",2842,"""[""sanket1729"", ""dr-orlovsky""]"""
"""funcx-faas/funcX""","""funcx-faas""","""funcX""",1188111034,747,"""Title: Mild clarification of d…",427,"""[""sirosen"", ""khk-globus""]"""


In [17]:
#SO we have just "usernames" that take part in "events" and "content", so I assume that the first name in "usernames" is the person
#who created the issue, and the last one who made the PR (although in practice thats the person who merged the PR)
#anyway in real case scenario we would have to find out which name comes from the author of Pull Request
#and would have to call the github API for data from that user
#to simplify the things let's just turn the user names into labels
#and join the labels with index of the kaggle dataset with users info, this way we can quickly mock the process of calling the Github repo for real user data

pr_df = pr_df.with_columns(pl.col("usernames").str.replace_all("\[","").str.replace_all("\]","").str.replace_all("\"","").str.split(by=","))

In [18]:
pr_df = pr_df.with_columns(users = pl.col("usernames").list.last())

In [19]:
pr_df["users"].value_counts(sort=True).shape

(176861, 2)

In [20]:
from sklearn.preprocessing import LabelEncoder

In [21]:
le = LabelEncoder()
pr_df = pr_df.with_columns(pl.col("users").map_batches(le.fit_transform).alias("mock_number"))
pr_df.head()

repo,parent_repo,child_repo,issue_id,issue_number,issue,text_size,usernames,users,mock_number
str,str,str,i64,i64,str,i64,list[str],str,i64
"""kaisermann/svelte-i18n""","""kaisermann""","""svelte-i18n""",550510104,40,"""Title: WIP - v3 username_0: """,2398,"[""kaisermann"", "" elbourki""]",""" elbourki""",52812
"""material-components/material-c…","""material-components""","""material-components-ios""",551064006,9444,"""Title: [AppBar] Fix swipe to g…",355,"[""jverkoey"", "" bryanoltman""]",""" bryanoltman""",38978
"""dlang/phobos""","""dlang""","""phobos""",551980198,7355,"""Title: Add initial support for…",4306,"[""wilzbach"", "" etcimon"", … "" jacob-carlborg""]",""" jacob-carlborg""",66742
"""patternfly/patternfly-elements""","""patternfly""","""patternfly-elements""",552466427,686,"""Title: added fix for nav prior…",1995,"[""starryeyez024"", "" LyndseyR""]",""" LyndseyR""",14002
"""NixOS/nixpkgs""","""NixOS""","""nixpkgs""",553379763,78253,"""Title: coqPackages.CoLoR: 1.4.…",2544,"[""vbgl"", "" jpas""]",""" jpas""",72876


In [ ]:
#TODO: join the pr_df with github users dataset from kaggle
#create a dataset of "available" Good First Issues - we can mock also this dataset
#instead of calling github API for current/open Good First Issues, we can just re-use the data from the datasets we used so far
#build model that will compare New Users to Old Users
#build model comparing Old Issues with Open Issues